In [128]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time

In [78]:
start_date = datetime.strptime("2023-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-05-21", "%Y-%m-%d")

date_list = []
current_date = start_date

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    date_list.append(formatted_date)
    current_date += timedelta(days=1)


['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-14', '2023-01-15', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-28', '2023-01-29', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-04', '2023-02-05', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-11', '2023-02-12', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-18', '2023-02-19', '2023-02-20', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-25', '2023-02-26', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-04', '2023-03-05', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-11', '2023-03-12', '2023

In [156]:
job_list = ['arrange', 'in', 'out', 'retin', 'retout', 'trans', 'SHOP_REQ']
# arrange 조정
# in 입고
# out 출고
# retin 반품입고
# retout 반품출고
# trans 배송처리
# SHOP_REQ 매장출고

In [157]:
def get_stock_log(start_date, end_date, job, page):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
    # start_date = '2023-01-01'
    # end_date = '2023-05-21'
    # job = 'in'
    limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_stock_tx_info&start_date={start_date}&end_date={end_date}&job={job}&page={page}&limit={limit}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError


In [126]:
def save_stock_log(cursor, data_list):

    query = """
        INSERT INTO "stock_log" (seq, crdate, product_id, job, qty, stock, bad, owner, memo, supply_code, supply_price, name, options, barcode, origin, brand, enable_sale, org_price, org_price_qty, order_seq, sheet, sheet_name, product_memo) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
        ON CONFLICT (seq) DO UPDATE 
        SET crdate = EXCLUDED.crdate, product_id = EXCLUDED.product_id, job = EXCLUDED.job, qty = EXCLUDED.qty, stock = EXCLUDED.stock, 
            bad = EXCLUDED.bad, owner = EXCLUDED.owner, memo = EXCLUDED.memo, supply_code = EXCLUDED.supply_code, 
            supply_price = EXCLUDED.supply_price, name = EXCLUDED.name, options = EXCLUDED.options, barcode = EXCLUDED.barcode, 
            origin = EXCLUDED.origin, brand = EXCLUDED.brand, enable_sale = EXCLUDED.enable_sale, org_price = EXCLUDED.org_price, 
            org_price_qty = EXCLUDED.org_price_qty, order_seq = EXCLUDED.order_seq, sheet = EXCLUDED.sheet, 
            sheet_name = EXCLUDED.sheet_name, product_memo = EXCLUDED.product_memo;
    """

    for d in data_list:
        cursor.execute(query, d)


In [191]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


for date in date_list:
    for job in job_list:
        page = 1
        data = ['data']
        while len(data) > 0:
            data = get_stock_log(date, date, job, str(page))
            data_list = [list(d.values()) for d in data]
            save_stock_log(cursor, data_list)
            # time.sleep(3)
            print(date + ' | ' + job + ' | ' + str(page))
            page += 1


conn.commit()
conn.close()

2023-01-01 | arrange | 1
2023-01-01 | in | 1
2023-01-01 | out | 1
2023-01-01 | retin | 1
2023-01-01 | retout | 1
2023-01-01 | trans | 1
2023-01-01 | SHOP_REQ | 1
2023-01-02 | arrange | 1
2023-01-02 | in | 1
2023-01-02 | out | 1
2023-01-02 | retin | 1
2023-01-02 | retout | 1
2023-01-02 | trans | 1
2023-01-02 | trans | 2
2023-01-02 | trans | 3
2023-01-02 | trans | 4
2023-01-02 | trans | 5
2023-01-02 | trans | 6
2023-01-02 | trans | 7
2023-01-02 | trans | 8
2023-01-02 | trans | 9
2023-01-02 | trans | 10
2023-01-02 | trans | 11
2023-01-02 | trans | 12
2023-01-02 | trans | 13
2023-01-02 | trans | 14
2023-01-02 | SHOP_REQ | 1
2023-01-03 | arrange | 1
2023-01-03 | in | 1
2023-01-03 | out | 1
2023-01-03 | retin | 1
2023-01-03 | retout | 1
2023-01-03 | trans | 1
2023-01-03 | trans | 2
2023-01-03 | trans | 3
2023-01-03 | trans | 4
2023-01-03 | trans | 5
2023-01-03 | trans | 6
2023-01-03 | trans | 7
2023-01-03 | SHOP_REQ | 1
2023-01-04 | arrange | 1
2023-01-04 | in | 1
2023-01-04 | out | 1
2023-0

In [187]:
type(data)

list

In [188]:
len(data)

1

In [184]:
test = get_stock_log('2023-01-01', '2023-05-01', 'trans', '1000')

In [176]:
data = ['test']

In [185]:
len(test)

0